## Download required data from yahoo finance

### Import libraries

In [2]:
import yfinance as yf
import pandas as pd

In [8]:
import asyncio
from twscrape import API, gather

### Download the stock data

In [50]:
# tickers is the list of stock tickers of 10 companies listed on the NYSE
tickers = ["AAPL", "AMZN", "MSFT", "GOOG", "NVDA", "NFLX", "DIS", "TSLA", "LLY", "BA"]
start_date = "2019-01-01"
end_date = "2023-12-31"
for ticker in tickers:
    # download the data from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date) 
    # convert the downloaded data into csv files
    data.to_csv("stock_data/{ticker}_{start_date[:4]}-{end_date[:4]}")
print("Data added to stock_data folder")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Data added to stock_data folder


## twitter data

In [4]:
api = API()

In [9]:
async def scrape_tweets(company, start_date, end_date):
    
    data = []  #create an empty list to be used to store the search results
    
    #define the search query. Include start date and end date
    q = f"${company} until:{end_date} since:{start_date} lang:en" 
    save_to_file = f"twitter_data/{company}_{start_date}-{end_date}"
    
    async for tweet in api.search(q, limit=300000): #iterate over the search results
        c = [tweet.id, tweet.date, tweet.rawContent, tweet.likeCount, tweet.retweetCount, tweet.user.location] #list of attributes to return 
        data.append(c)  #add each new list of attributes to 'data'
    
    df = pd.DataFrame(data, columns=['Tweet_ID', 'Time_Created', 'Text', 'Likes', 'Retweets', 'Location']) #convert the list to a dataframe
    df.to_csv(save_to_file, index = False) #save to a chosen directory on the computer
    return df

In [20]:
nvidia_tweets = await scrape_tweets("NVDA", "2024-01-18", "2024-01-20")
nvidia_tweets

2024-04-21 20:13:46.611 | WARNING  | twscrape.queue_client:_check_rep:157 - Ban detected: 200 - 49/50 - testid1106 - (326) Authorization: Denied by access control: To protect our users from spam and other malicious activity, this account is temporarily locked. Please log in to https://twitter.com to unlock your account.
2024-04-21 20:15:12.025 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:275 - No account available for queue "SearchTimeline". Next available at 20:27:12
2024-04-21 20:17:27.429 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:281 - Continuing with account testid1106 on queue SearchTimeline
2024-04-21 20:18:06.769 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:275 - No account available for queue "SearchTimeline". Next available at 20:27:12
2024-04-21 20:27:13.258 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:281 - Continuing with account divij_jasuja on queue SearchTimeline
2024-04-21 20:27:58.720 | INFO     | twscrape.accounts

,Tweet_ID,Time_Created,Text,Likes,Retweets,Location
0,1748495382944895066,2024-01-19 23:59:19+00:00,"Chip names, chip games! Which stock has a high...",0,0,"Pittsburgh, PA"
1,1748495348174369118,2024-01-19 23:59:11+00:00,"Best place for day trading, swing trading, sto...",0,0,
2,1748495299641848185,2024-01-19 23:58:59+00:00,Celebrate with us these amazing profits on $NV...,11,2,"Texas, USA"
3,1748495275553960360,2024-01-19 23:58:53+00:00,@BradMunchen Talking “distressed” I think is e...,0,0,Canada
4,1748495072960631288,2024-01-19 23:58:05+00:00,First Majestic Stock (NYSE:AG) Will Shine in a...,2,0,USA
...,...,...,...,...,...,...
6466,1747775029905924363,2024-01-18 00:16:53+00:00,"Canceled Cybertruck which is DEAD, and placed ...",2,0,Portugal
6467,1747771556976025982,2024-01-18 00:03:05+00:00,🚨 Quote of the Day 🚨 “The pessimist sees diffi...,1,0,Makin Money
6468,1747771417226080466,2024-01-18 00:02:32+00:00,"Thurs Jan 18 Watchlist\n\n$SPY C > 474, 467 / ...",0,0,
6469,1747771167883059323,2024-01-18 00:01:33+00:00,@The_AI_Investor Clearly that person has been ...,7,0,Boston | Cape Cod


In [21]:
from datetime import date
# nvidia_tweets[nvidia_tweets['Time_Created'] < date(2024, 1, 8) or nvidia_tweets['Time_Created'] > date(2024, 1, 15)]
# pd.to_datetime(nvidia_tweets['Time_Created']).dt.date
# nvidia_tweets = pd.read_csv("twitter_data/NVDA_2024-01-08-2024-01-15")
nvidia_tweets['Time_Created'] = pd.to_datetime(nvidia_tweets['Time_Created']).dt.date
nvidia_tweets['Time_Created'].sort_values().value_counts()[:15]


Time_Created
2024-01-19    3287
2024-01-18    2945
2024-01-17      64
2024-01-16      27
2024-01-09      20
2024-01-08      18
2024-01-11      13
2024-01-10       9
2024-01-12       8
2024-01-15       5
2024-01-05       4
2024-01-14       4
2023-05-31       3
2023-12-29       3
2023-12-21       3
Name: count, dtype: int64